In [1]:
!pip install tensorflow==2.3.1
!pip install q keras==2.4.3
!pip install esprima

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from keras.models import Sequential
from keras import layers
from keras.backend import clear_session
from keras.preprocessing.text import Tokenizer
from keras.layers import Activation, Dense, Reshape 
import numpy as np
from keras import backend as K
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout, Conv1D, GlobalMaxPooling1D
from keras.models import load_model
from numpy import asarray
from numpy import save
from numpy import load
from sklearn.metrics import classification_report , confusion_matrix
import esprima as esp
import re
from keras.initializers import Constant
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy
from keras.utils import plot_model
from keras import Model
from keras.utils import Sequence
import pydot
from nltk.tokenize import word_tokenize
import pickle
from keras import backend as K
import os
import random
import re
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import logging
from time import time
from functools import partial
import concurrent.futures
from threading import current_thread
from tqdm import tqdm
import gc
from keras.layers import Input
from keras.models import Model , clone_model
np.random.seed(123)
plt.style.use('ggplot')

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Hyperparameter for Our Neural Network

In [5]:
vocab_size = 100000
num_words = 400000
maxlen = 1024
embedding_dim = 50
myoptimizer = 'adam'
myloss= 'binary_crossentropy'
mymetrics = 'accuracy'

save_object & load_object to save and load All objects we are need it

In [6]:
def save_object(obj, filename):
    """
    _ INPUT (obj) THE OBJECT WE NEED SAVW IT (filename) THE NAME OF OBJECT
    """
    filename = os.path.join('/content/drive/MyDrive/Colab Notebooks/results/par',filename)
    with open(filename+".pkl", 'wb') as outp:
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)
    outp.close()

In [7]:
def load_object(filename):
    """
    _ INPUT THE NAME OF OBJECT WE NEED LOAD IT
    """
    filename = os.path.join('/content/drive/MyDrive/Colab Notebooks/results/par',filename)
    with open(filename+".pkl", 'rb') as outp:
        loaded_object = pickle.load(outp)
    outp.close()
    return loaded_object

In [8]:
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

lowering_remove_url_ip_: this function used to cleaning js scripts 


1.   remove urls
2.   remove ips
3.   remove comments
4.   convert to lower case



In [9]:
def lowering_remove_url_ip_(script):
    """
    _ INPUT IS JS FILE 
    _ OUTPUT IS CLEAN JS FILE
    """
    script = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', script)
    script = re.sub(r'((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)', '', script)
    script = re.sub(r"(/\*([^*]|[\r\n]|(\*+([^*/]|[\r\n])))*\*+/)|(//.*)",'',script)
    script = script.lower()
    return script

preprocessing_dataset_: this function used to tokenzation js file and convert text to sequences

In [10]:
def preprocessing_dataset_(scripts,fit = True):
    """
    _INPUT (scripts) JS FILES (fit) IF THE FILES IS FROM TRANING SET IT WILL BE (fit=True) ELSE IT WILL BE (fit=False)
    """
    if fit:
        tokenizer = Tokenizer(num_words=num_words)
        tokenizer.fit_on_texts(scripts)
        our_tokenizer = save_object(tokenizer, "our_tokenizer")
    else :
        tokenizer = load_object("our_tokenizer")
    X = tokenizer.texts_to_sequences(scripts)
    X = pad_sequences(X, padding='post', maxlen=maxlen)
    return X

sequence_of_syntax_units: this function used to convert java script code to the abstract syntax tree then extract the sequences of syntax units with detailed information

In [11]:
def sequence_of_syntax_units(scripts):
    """
    sctipts : list or single javascript code 
    return sequence of syntax units
    each syntax unit correspond to line in an abstract syntax tree
    """
    if type(scripts) is list:
        # esp.parseScript(script) returns abstract syntax tree of each js scripts
        return [re.sub('\s+', ' ', ''.join(str(esp.parseScript(script)).split('\n'))) for script in scripts]
    elif type(scripts) is str:
        return re.sub('\s+', ' ', ''.join(str(esp.parseScript(scripts)).split('\n')))
    else:
        raise ValueError('The type of scripts parameter must be {list or string}')

In [12]:
# create word tokens
def sequence_of_syntax_units_to_tokens(seq_of_syn_units):
    """
    Convert Sequence of syntax units to tokens to train  word2vec model 
    -----------------------------------------
    seq_of_syn_units: list of sequence syntax units correspond to abstract syntax trees
    """
    return list(map(word_tokenize, seq_of_syn_units))

feature_selection: this function used to make initial feature selection on chunk of js file after used the preprocessing_dataset_ function

In [14]:
def feature_selection(x_chunk,y_chunk):
    """
    _INPUT (x_chunk) SET OF JS CODE AFTER APPLY PREPROCESSING DATASET FUNCTION (y_chunk)
    """
    indexes = random.sample(range(3, x_chunk.shape[0]), random.randint(1, x_chunk.shape[0]-3))
    x_chunk[indexes,:,:] = 0
    return x_chunk,y_chunk

evaluation: this function used to evaluate our model and save result in comparison table 

In [16]:
def evaluation(model,chunk_number,history,X_train,y_train,X_test,y_test,model_comparison_table):
    """
    _INPUT 
    (model) THE MODEL THAT WE NEED EVALUATE
    (chunk_number) NUMBER OF CHUNK FROM DATA WE PROCESS
    (history) IF WE NEED CALL FUNCTION plot_history
    (X_train,y_train,X_test,y_test) TRAINING DATA SET AND TESTING DATA SET
    (model_comparison_table) DECTIONARY TO SAVE RESULT [train_loss, train_accuracy,test_loss, test_accuracy,Classification_report]
    """
    train_loss, train_accuracy = model.evaluate(X_train, y_train, verbose=False)
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=False)
    predicted = model.predict(X_test)
    Classification_report = classification_report(np.round(model.predict(X_test)), y_test)
    inputer(model_comparison_table,chunk_nuber,train_loss, train_accuracy,test_loss, test_accuracy,str(Classification_report))

create_model: this function used to creat neural network with 11 layer or with 9 layer

In [17]:
def create_model(shape=None,down=False):
    """
    _INPUT (down) IF WE WANT TO CREAT MODEL WITHOUT FIRST LSTM LAYER  (shape) IF THE PREVIOS PAREMETER IS TRUE WE NEED DIFINE THE SHAPE FOR INPUT LAYER
    _OUTPUT THE MODEL 
    """
    model = Sequential() 
    if down:
        model.add(layers.InputLayer(input_shape=shape))
        model.add(Bidirectional(LSTM(units=50, input_shape=(None, 50), return_sequences=True)))
        # TextCNN with 4 conv layers
        model.add(Conv1D(128, 7, activation='tanh', input_shape=(None, 32)))
        model.add(Conv1D(128, 15, activation='tanh'))
        model.add(Conv1D(128, 25, activation='tanh'))
        model.add(Conv1D(128, 35, activation='tanh'))
        model.add(GlobalMaxPooling1D())
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(1, activation='sigmoid'))
    else:
        model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
        model.add(Bidirectional(LSTM(units=50, input_shape=(None, 50), return_sequences=True))) 
        model.add(Bidirectional(LSTM(units=50, input_shape=(None, 50), return_sequences=True)))
        # TextCNN with 4 conv layers
        model.add(Conv1D(128, 7, activation='tanh', input_shape=(None, 32)))
        model.add(Conv1D(128, 15, activation='tanh'))
        model.add(Conv1D(128, 25, activation='tanh'))
        model.add(Conv1D(128, 35, activation='tanh'))
        model.add(GlobalMaxPooling1D())
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=myoptimizer,loss=myloss,metrics=[mymetrics])
#     model.summary()
    return model

fit_model: this function used to fit our models with 10 epochs 

In [19]:
def fit_model(model,X_train,y_train,X_validation,y_validation):
    """
    _INPUT (model) (X_train,y_train,X_validation,y_validation)
    """
    history = model.fit(X_train, y_train,epochs=10,verbose=False,validation_data=(X_validation, y_validation))
    return model , history

creat_metrics: this function used to creat dectionary to save result for evaluation

In [20]:
def creat_metrics():
    model_comparison_table = {}
    model_comparison_table['chunk_nuber'] = []
    model_comparison_table['train_loss'] = []
    model_comparison_table['train_accuracy'] = []
    model_comparison_table['test_loss'] = []
    model_comparison_table['test_accuracy'] = []
    model_comparison_table['classification_report'] = []
    return model_comparison_table

inputer: this function used to Fill dictionary with result evaluate the model

In [21]:
def inputer(model_comparison_table,chunk_nuber,train_loss, train_accuracy,test_loss,test_accuracy,classification_report):#precision_score,recall_score,f1_score):
    model_comparison_table['chunk_nuber'].append(str(chunk_nuber))
    model_comparison_table['train_loss'].append(train_loss)
    model_comparison_table['train_accuracy'].append(train_accuracy)
    model_comparison_table['test_loss'].append(test_loss)    
    model_comparison_table['test_accuracy'].append(test_accuracy)  
    model_comparison_table['classification_report'].append(classification_report)    

chunking_data: this function used to split chunk to train & test & validation

In [22]:
def chunking_data(Data,validation_size=0.10,test_size=0.20,train_size=0.70):
    """
    _INPUT (Data) CHUNK OF DATASET (validation_size=0.10,test_size=0.20,train_size=0.70) SIZE OF SPLITNG
    _OUTPUT (X_train , y_train , X_test , y_test , X_validation , y_validation)
    """
    test_data = Data.iloc[:int((Data.shape[0]*test_size))]
    validation_data = Data.iloc[int((Data.shape[0]*test_size)):(int((Data.shape[0]*validation_size))+int((Data.shape[0]*test_size)))]
    train_data = Data.iloc[(int((Data.shape[0]*test_size))+int((Data.shape[0]*validation_size))):]
    X_test = test_data[test_data.columns[-2]]
    y_test = test_data[test_data.columns[-1]]
    X_validation = validation_data[validation_data.columns[-2]]
    y_validation = validation_data[validation_data.columns[-1]]
    X_train = train_data[train_data.columns[-2]]
    y_train = train_data[train_data.columns[-1]]
    return X_train , y_train , X_test , y_test , X_validation , y_validation

read__script: this function use to read path script from dataframe then read the js file by this path and finally call sequence_of_syntax_units on this file

In [23]:
def read__script(script_paths,parent_path,y):
    x_temp = []
    y_temp = []
    todrop = []
    for path in script_paths:
        try:
            with open(os.path.join(parent_path,path), 'r') as js_file: # , encoding='utf-8'
                script = js_file.read()
            js_file.close()
#             script = lowering_remove_url_ip_(script)
            script = sequence_of_syntax_units(script)
            new_y = y[script_paths.index(path)]
            x_temp.append(script)
            y_temp.append(new_y)
        except:
            pass
    if len(x_temp)>0:
        return x_temp , np.array(y_temp)
    else:
        return None , None

In [ ]:
model_result_table = creat_metrics()
bottom_model_result_table = creat_metrics()
model_is_created = False
model_tow_is_created = False
for i , df_iterator in enumerate(pd.read_csv('/content/drive/MyDrive/Colab Notebooks/XSS_DataSet.csv',chunksize=2000)):
    try:
      X_train , y_train , X_test , y_test , X_validation , y_validation = chunking_data(df_iterator)
      X_train , y_train = read__script(X_train.tolist(),"/content/drive/MyDrive/Colab Notebooks",y_train.tolist())
      X_test , y_test = read__script(X_test.tolist(),"/content/drive/MyDrive/Colab Notebooks/",y_test.tolist())
      X_validation , y_validation = read__script(X_validation.tolist(),"/content/drive/MyDrive/Colab Notebooks/",y_validation.tolist())
      if X_train!=None:
          X_train = preprocessing_dataset_(X_train , fit = True )
          X_test = preprocessing_dataset_(X_test , fit = False )
          X_validation = preprocessing_dataset_(X_validation , fit = False )
          if not model_is_created:
              model = create_model()
              model.save("/content/drive/MyDrive/Colab Notebooks/results/models/model/model chunk "+str(i)+".h5")
              model_is_created = True
          else:
              model = load_model("/content/drive/MyDrive/Colab Notebooks/results/models/model/model chunk "+str(i-1)+".h5")
          model , history_Model = fit_model(model,X_train,y_train,X_validation,y_validation)
          top_model = Model(inputs=model.input,outputs=model.layers[1].output)
          evaluation(model,i,history_Model,X_train,y_train,X_test,y_test,model_result_table)
          model.save("/content/drive/MyDrive/Colab Notebooks/results/models/model/model chunk "+str(i)+".h5")
          keras_function_train = K.function([model.input], [model.layers[1].output])
          X_train = keras_function_train([X_train, 1])[0]
          X_validation = top_model.predict(X_validation)
          X_test = top_model.predict(X_test)
          X_train , y_train = feature_selection(X_train,y_train)
          if not model_tow_is_created:
              bottom_model = create_model(shape=X_train[0].shape,down=True)
              bottom_model.save("/content/drive/MyDrive/Colab Notebooks/results/models/bottom_model/bottom_model chunk "+str(i)+".h5")
              model_tow_is_created = True 
          else:
              bottom_model = load_model("/content/drive/MyDrive/Colab Notebooks/results/models/bottom_model/bottom_model chunk "+str(i-1)+".h5")
          bottom_model , history_Model_bottom_model = fit_model(bottom_model,X_train,y_train,X_validation,y_validation)
          evaluation(bottom_model,i,history_Model_bottom_model,X_train,y_train,X_test,y_test,bottom_model_result_table)
          bottom_model.save("/content/drive/MyDrive/Colab Notebooks/results/models/bottom_model/bottom_model chunk "+str(i)+".h5")
    except:
        print("error")

0it [00:00, ?it/s]

(784, 1024)
(230, 1024)
(110, 1024)
(110, 1024)
(110,)
(110, 1024, 100)
(230, 1024, 100)
(784, 1024, 100)
(784,)


1it [09:17, 557.65s/it]

(791, 1024)
(226, 1024)
(110, 1024)
(110, 1024)
(110,)
(110, 1024, 100)
(226, 1024, 100)
(791, 1024, 100)
(791,)


3it [35:11, 717.61s/it]

error


In [ ]:
save_object(model_result_table, "model_result_table")
save_object(bottom_model_result_table, "bottom_model_result_table")

In [ ]:
df = pd.DataFrame(model_result_table)
df_ = pd.DataFrame(bottom_model_result_table)
df_.to_csv('/content/drive/MyDrive/Colab Notebooks/results/models/bottom_model_result_table.csv',index=False)
df.to_csv('/content/drive/MyDrive/Colab Notebooks/results/models/model_result_table.csv',index=False)
df.head(5)

In [ ]:
df_.head(5)

In [ ]:
for i in df['classification_report']:
    print("================================")
    print(i)

In [ ]:
for i in df_['classification_report']:
    print("================================")
    print(i)